In [0]:
spark.conf.set(
  "fs.azure.account.key.snowflakesession.blob.core.windows.net",
  "QFOWMa+1PeHMd8oTgZqPeddxMAzLv1qlIO0qdoeUN2lg1qYlMHlfDQUCvLju+ic/i9RnJSyadnch+ASts7qAeQ=="
)
df_raw = spark.read \
    .option("multiLine", "true") \
    .json("wasbs://adf@snowflakesession.blob.core.windows.net/application_logs.json")

df_raw.printSchema()
df_raw.show(truncate=False)


root
 |-- api_endpoint: string (nullable = true)
 |-- error_message: string (nullable = true)
 |-- event_id: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- log_level: string (nullable = true)
 |-- region: string (nullable = true)
 |-- response_time_ms: long (nullable = true)
 |-- service_name: string (nullable = true)
 |-- user_id: string (nullable = true)

+--------------+-------------+--------+--------------------+---------+------+----------------+-------------+-------+
|api_endpoint  |error_message|event_id|event_time          |log_level|region|response_time_ms|service_name |user_id|
+--------------+-------------+--------+--------------------+---------+------+----------------+-------------+-------+
|/api/v1/orders|Timeout      |evt_101 |2025-09-10T10:15:30Z|ERROR    |IN    |850             |order-service|U1001  |
+--------------+-------------+--------+--------------------+---------+------+----------------+-------------+-------+



In [0]:
from pyspark.sql.functions import col, when, to_timestamp

df_transformed = (
    df_raw
    .withColumn("EVENT_TIME", to_timestamp(col("event_time")))
    .withColumn(
        "ERROR_FLAG",
        when(col("log_level") == "ERROR", 1).otherwise(0)
    )
    .select(
        col("event_id").alias("EVENT_ID"),
        col("EVENT_TIME"),
        col("service_name").alias("SERVICE_NAME"),
        col("log_level").alias("LOG_LEVEL"),
        col("user_id").alias("USER_ID"),
        col("api_endpoint").alias("API_ENDPOINT"),
        col("response_time_ms").alias("RESPONSE_TIME_MS"),
        col("error_message").alias("ERROR_MESSAGE"),
        col("region").alias("REGION"),
        col("ERROR_FLAG")
    )
)

df_transformed.show()


+--------+-------------------+-------------+---------+-------+--------------+----------------+-------------+------+----------+
|EVENT_ID|         EVENT_TIME| SERVICE_NAME|LOG_LEVEL|USER_ID|  API_ENDPOINT|RESPONSE_TIME_MS|ERROR_MESSAGE|REGION|ERROR_FLAG|
+--------+-------------------+-------------+---------+-------+--------------+----------------+-------------+------+----------+
| evt_101|2025-09-10 10:15:30|order-service|    ERROR|  U1001|/api/v1/orders|             850|      Timeout|    IN|         1|
+--------+-------------------+-------------+---------+-------+--------------+----------------+-------------+------+----------+



In [0]:
sfOptions = {
    "sfURL": "FJPKTEI-SZ94461.snowflakecomputing.com",
    "sfUser": "LOG_ANALYTICS_USER",
    "sfPassword": "Log@1234",
    "sfWarehouse": "LOG_ANALYTICS_WH",
    "sfDatabase": "LOG_ANALYTICS_DB",
    "sfSchema": "LOG_SCHEMA"
}
